In [5]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

COLUMNAS_GRD = [
    "RUT",
    "SEXO",
    # "EDAD_CANT",
    "PREVI",
    "BENEF",
    "NOM_HOPI_P",
    "FECHA_INGRESO",
    "FECHA_EGRESO",
    "ANO_EGR",
    "DIAS_ESTAD",
    "DIAG1",
    "DIAG2",
    "DIAG3",
    "INTERV_Q",
    "COD_FONASA",
    "INTERV_Q_PPAL",
    "INTERV_Q_2",
    "INTERV_Q_3",
    "T_DIAG1",
    "T_DIAG2",
    "T_DIAG3",
    "PROCED",
    "PROCED_PPAL",
    "PROCED_2",
    "PROCED_3",
]

COLUMNA_GRD_INTERNO = [
    "rut",
    "sexo_(cod)",
    "prevision_(cod)",
    "hospital_de_procedencia_(cod)",
    "fecha_ingreso",
    "fecha_de_egreso",
    "estancias_totales",
    "diag_01_principal_(cod)",
    "diag_01_principal_(des)",
    "diag_02_secundario_(cod)",
    "diag_02_secundario_(des)",
    "diag_03_secundario_(cod)",
    "diag_03_secundario_(des)",
    "proced_01_principal_(cod)",
    "proced_01_principal_(des)",
]

In [18]:
# Lee GRD del INT
df_sabanas = pd.read_csv("../data/processed/grd_sabanas.csv", usecols=COLUMNAS_GRD)
# Formatea fechas de GRD
df_sabanas["FECHA_INGRESO"] = pd.to_datetime(df_sabanas["FECHA_INGRESO"])
df_sabanas["FECHA_EGRESO"] = pd.to_datetime(df_sabanas["FECHA_EGRESO"])

# Lee GRD Interno del INT
df_grd_interno = pd.read_csv("../data/processed/grd_interno.csv", usecols=COLUMNA_GRD_INTERNO)

# Lee la base de pabellon del INT
df_pabellon = pd.read_csv(
    "../data/external/datos_limpios_pabellon.csv", encoding="latin-1", sep=";"
)
# Formatea fechas de Pabellon
df_pabellon["fecha"] = pd.to_datetime(df_pabellon["fecha"])

# Formatea duraciones de operaciones
df_pabellon["duracion"] = pd.to_timedelta(df_pabellon["duracion"])

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_87913/3707223252.py:2: DtypeWarning: Columns (80,156) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sabanas = pd.read_csv("../data/processed/grd_sabanas.csv", usecols=COLUMNAS_GRD)


In [19]:
# Limpia la base de pabellon, sacando las suspendidas y sin duracion
df_pabellon_limpio = df_pabellon.copy()
df_pabellon_limpio = df_pabellon_limpio.query("suspendida == 'NO' and duracion.notna()")

In [20]:
# Une la base de GRD y Pabellon segun el RUT del Paciente
grd_y_pabellon = df_sabanas.merge(
    df_pabellon_limpio, how="inner", left_on="RUT", right_on="ID_PACIENTE"
)

# Filtra la union de GRD y Pabellon para dejar las Int. Q que se hicieron dentro del egreso
grd_y_pabellon = grd_y_pabellon.query("fecha >= FECHA_INGRESO and fecha <= FECHA_EGRESO")

In [21]:
resumen_duraciones = (
    grd_y_pabellon.groupby(["ANO_EGR", "DIAG1"])["duracion"].describe()
    .reset_index()
)

In [22]:
resumen_duraciones

,ANO_EGR,DIAG1,count,mean,std,min,25%,50%,75%,max
0,2015,A156,3,0 days 02:13:20,0 days 00:32:31.922129594,0 days 01:40:00,0 days 01:57:30,0 days 02:15:00,0 days 02:30:00,0 days 02:45:00
1,2015,A418,3,0 days 01:31:40,0 days 00:16:04.365076099,0 days 01:20:00,0 days 01:22:30,0 days 01:25:00,0 days 01:37:30,0 days 01:50:00
2,2015,B206,2,0 days 00:45:00,0 days 00:00:00,0 days 00:45:00,0 days 00:45:00,0 days 00:45:00,0 days 00:45:00,0 days 00:45:00
3,2015,B217,1,0 days 01:30:00,NaT,0 days 01:30:00,0 days 01:30:00,0 days 01:30:00,0 days 01:30:00,0 days 01:30:00
4,2015,B218,2,0 days 02:50:00,0 days 01:24:51.168824543,0 days 01:50:00,0 days 02:20:00,0 days 02:50:00,0 days 03:20:00,0 days 03:50:00
...,...,...,...,...,...,...,...,...,...,...
1705,2022,U071,8,0 days 00:56:15,0 days 00:22:57.108772547,0 days 00:25:00,0 days 00:42:30,0 days 00:50:00,0 days 01:16:15,0 days 01:30:00
1706,2022,Z430,1,0 days 01:55:00,NaT,0 days 01:55:00,0 days 01:55:00,0 days 01:55:00,0 days 01:55:00,0 days 01:55:00
1707,2022,Z450,126,0 days 01:00:54.761904761,0 days 00:26:54.613620299,0 days 00:25:00,0 days 00:46:15,0 days 00:55:00,0 days 01:05:00,0 days 04:35:00
1708,2022,Z470,10,0 days 02:19:30,0 days 00:25:32.644772933,0 days 01:45:00,0 days 01:55:00,0 days 02:20:00,0 days 02:43:45,0 days 02:50:00
